# <b>Projeto Integrador IV-A</b>
**Objetivo:**<br>
Demonstrar o uso do Pycuda, Rapids e Numba para processamento de imagens.<br>

**Por**:Vinícius Guimarães

In [1]:
#verificar se a GPU disponível é uma T4, P4 ou P100 antes de instalar as dependências necessárias
!nvidia-smi

Wed Jan 11 13:19:43 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#instalar o pycuda, caso já não esteja instalado
!pip install pycuda
#instalar dependencias necessarias para o RapidsAI
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/env-check.py
# Atualizar o colab e reiniciar o kernel
!bash rapidsai-csp-utils/colab/update_gcc.sh
import os
os._exit(00)

In [1]:
import os
# Instalação do RapidsAI, demora um tempo considerável para finalizar...
!python rapidsai-csp-utils/colab/install_rapids.py stable
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'
os.environ['CONDA_PREFIX'] = '/usr/local'
#Instalação do condacolab, também irá reiniciar o kernel
import condacolab
condacolab.install()

Found existing installation: cffi 1.15.1
Uninstalling cffi-1.15.1:
  Successfully uninstalled cffi-1.15.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 446.7/446.7 KB 13.0 MB/s eta 0:00:00
Installing RAPIDS Stable 22.12
Starting the RAPIDS install on Colab.  This will take about 15 minutes.
/bin/sh: 1: conda: not found
RAPIDS conda installation complete.  Updating Colab's libraries...
⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:15
🔁 Restarting kernel...


In [1]:
#A VM irá reiniciar pelo menos três vezes durante a instalação dos itens 
# essenciais básicos, e após isso, rodar esse bloco com as dependencias

import condacolab
condacolab.check()
import numba

import pycuda.driver as drv
import cv2 as cv
import pycuda.gpuarray as gpuarray
import pycuda.autoinit
from pycuda.compiler import SourceModule
from skimage import io
from PIL import Image 
from numba import cuda
import matplotlib.pylab as plt

dev = drv.Device(0)
cuda.select_device(0)
drv.init()
drv.Device(0).retain_primary_context()

✨🍰✨ Everything looks OK!


In [2]:
from time import time
from contextlib import contextmanager

@contextmanager
def timer(name):
  t0 = time()
  yield
  print(f'[{name}] done in {time() - t0:.2f} s')

In [ ]:
print("Device: %s", dev.name())
print(" Compute Capability: %d.%d" % dev.compute_capability())
print(" Total Memory: %s KB" % (dev.total_memory()//(1024)))
atts = [(str(att), value)
 for att, value in dev.get_attributes().items()]
atts.sort()
 
for att, value in atts:
 print(" %s: %s" % (att, value))

In [5]:
#importação do arquivo CSV com os links

import cudf
import dask_cudf
import io, requests

# download CSV file from GitHub
url='https://raw.githubusercontent.com/vinissaurus/pycuda_vision/main/data_sources/faceexp-comparison-data-test-public.csv'
content = requests.get(url).content.decode('utf-8')

# read CSV from memory 
with timer('cuDF'):
  faces_df = cudf.DataFrame()
  faces_df = cudf.read_csv(io.StringIO(content), usecols=[0], names=['urls'] , sep="," , quotechar='"')
  #faces_df = faces_df.astype(str)

  
#urls = dask_cudf.from_cudf(df, npartitions=2)

ModuleNotFoundError: ignored

In [ ]:
#starting_point = str(faces_df.at[59,'urls'])

import functools
import os

from matplotlib import gridspec
import matplotlib.pylab as plt
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

print("TF Version: ", tf.__version__)
print("TF Hub version: ", hub.__version__)
print("Eager mode enabled: ", tf.executing_eagerly())
print("GPU available: ", tf.config.list_physical_devices('GPU'))


def crop_center(image):
  """Returns a cropped square image."""
  shape = image.shape
  new_shape = min(shape[1], shape[2])
  offset_y = max(shape[1] - shape[2], 0) // 2
  offset_x = max(shape[2] - shape[1], 0) // 2
  image = tf.image.crop_to_bounding_box(
      image, offset_y, offset_x, new_shape, new_shape)
  return image

@functools.lru_cache(maxsize=None)
def load_image(image_url, image_size=(256, 256), preserve_aspect_ratio=True):
  """Loads and preprocesses images."""
  # Cache image file locally.
  image_path = tf.keras.utils.get_file(os.path.basename(image_url)[-128:], image_url)
  # Load and convert to float32 numpy array, add batch dimension, and normalize to range [0, 1].
  img = tf.io.decode_image(
      tf.io.read_file(image_path),
      channels=3, dtype=tf.float32)[tf.newaxis, ...]
  img = crop_center(img)
  img = tf.image.resize(img, image_size, preserve_aspect_ratio=True)
  return img

def show_n(images, titles=('',)):
  n = len(images)
  image_sizes = [image.shape[1] for image in images]
  w = (image_sizes[0] * 6) // 320
  plt.figure(figsize=(w * n, w))
  gs = gridspec.GridSpec(1, n, width_ratios=image_sizes)
  for i in range(n):
    plt.subplot(gs[i])
    plt.imshow(images[i][0], aspect='equal')
    plt.axis('off')
    plt.title(titles[i] if len(titles) > i else '')
  plt.show()

TF Version:  2.9.2
TF Hub version:  0.12.0
Eager mode enabled:  True
GPU available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
def load_random_image():
  try:
    rand_index = np.random.randint(0,50000)
    load_image(faces_df.at[rand_index,'urls'])
    return faces_df.at[rand_index,'urls']
  except:
    return load_random_image() #the perks of using recursion

#@vectorize(['Float'],target='cuda')
def image_fusion():
  #content_image_url = load_random_image()
  #style_image_url = load_random_image()
  output_image_size = 384

  content_img_size = (output_image_size, output_image_size)

  style_img_size = (384, 384) 

  content_image = load_image(load_random_image(), content_img_size)
  style_image = load_image(load_random_image(), style_img_size)
  style_image = tf.nn.avg_pool(style_image, ksize=[3,3], strides=[1,1], padding='SAME')

  # Load TF Hub module.

  hub_handle = 'https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2'
  hub_module = hub.load(hub_handle)

  # Stylize content image with given style image.
  # This is pretty fast within a few milliseconds on a GPU.

  outputs = hub_module(tf.constant(content_image), tf.constant(style_image))
  stylized_image = outputs[0]

  #show_n([content_image, style_image, stylized_image], titles=['Img A', 'Img B', 'Result'])

  return stylized_image

#@vectorize(['float32(float32,float32)'],target='cuda')
def art_fusion(img1, img2):
  # Load TF Hub module.

  hub_handle = 'https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2'
  hub_module = hub.load(hub_handle)

  # Stylize content image with given style image.
  # This is pretty fast within a few milliseconds on a GPU.

  outputs = hub_module(tf.constant(img1), tf.constant(img2))
  stylized_image = outputs[0]

  #show_n([content_image, style_image, stylized_image], titles=['Img A', 'Img B', 'Result'])
  
  return stylized_image
  

In [ ]:
def generate_art():
  with tf.device('/device:GPU:0'):
    art1 = image_fusion()
    art2 = image_fusion()
  
    result1 = art_fusion(art1, art2)

    show_n([art1, art2, result1], titles=['Img A', 'Img B', 'Result'])

def generate_art_w():
  with tf.device('/cpu:0'):
    art1 = image_fusion()
    art2 = image_fusion()

    result1 = art_fusion(art1, art2)

    show_n([art1, art2, result1], titles=['Img A', 'Img B', 'Result'])


In [ ]:
with tf.device('/device:GPU:0'):
  art1 = image_fusion()
  art2 = image_fusion()

  show_n([art1,art2])

NameError: ignored

In [ ]:
#fusion = art_fusion(art1,art2)

with tf.device('/device:GPU:0'):
  names=['A+B','A-B','A^B','B^A','A/B','B/A','A%B','B%A', '-A+B', 'A²+B²']
  result = [0]*12
  result[0] = art1+art2#art_fusion(art1,art2)
  result[1] = art1-art2
  result[2] = art1**art2
  result[3] = art2**art1
  result[4] = art1/art2
  result[5] = art2/art1
  result[6] = art1%art2
  result[7] = art2%art1
  result[8] = result[6]/result[7]
  result[9] = result[7]/result[6]
  result[10] = np.sqrt(art1) - art2**2
  result[11] = art_fusion(art1,art2)

  show_n(result,names)

In [ ]:
with timer('GPU processed art:'):
  generate_art()

with timer('CPU processed art:'):
  generate_art_w()

In [ ]:
from google.colab import files

source = files.upload()
style = files.upload()

In [ ]:
def custom_image_fusion(imgA, imgB):
  #content_image_url = load_random_image()
  #style_image_url = load_random_image()
  output_image_size = 384

  content_img_size = (output_image_size, output_image_size)

  style_img_size = (384, 384) 
  #source = files.upload()
  #style = files.upload()
  content_image = load_image(crop_center(imgA), content_img_size)
  style_image = load_image(crop_center(imgB), style_img_size)
  style_image = tf.nn.avg_pool(style_image, ksize=[3,3], strides=[1,1], padding='SAME')

  # Load TF Hub module.

  hub_handle = 'https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2'
  hub_module = hub.load(hub_handle)

  # Stylize content image with given style image.
  # This is pretty fast within a few milliseconds on a GPU.

  outputs = hub_module(tf.constant(content_image), tf.constant(style_image))
  stylized_image = outputs[0]

  #show_n([content_image, style_image, stylized_image], titles=['Img A', 'Img B', 'Result'])

  return stylized_image

In [ ]:
source = files.upload()
style = files.upload()